# 1번

In [ ]:
import re

def solution(new_id):
    #1단계
    new_id = new_id.lower()
    
    #2단계
    new_id = re.sub(r'[^a-z0-9._-]','', new_id)
    
    #3단계
    new_id = re.sub(r'[.]+', '.', new_id)
    
    #4단계
    new_id = new_id.strip('.')
    
    #5단계
    if new_id == '':
        new_id = 'a'
    
    #6단계
    if len(new_id) >= 16:
        new_id = new_id[:15]
        new_id = new_id.strip('.')
    
    #7단계
    last_char = new_id[-1]
    while len(new_id) <= 2:
        new_id += last_char
        
    return new_id

# 2번

In [ ]:
import itertools

def alphabetize(string):
    return ''.join(sorted(string))

def findsubsets(S,m):
    subset = set(itertools.combinations(S, m))
    subset = [ ''.join(x) for x in subset]
    return subset

def solution(orders, course):
    order_count = dict()
    longest_length = { x:0 for x in course}
    for order in orders:
        for length in course:
            if len(order) >= length:
                order = alphabetize(order)
                candidates = findsubsets(order, length)
                for candidate in candidates:
                    count = 0
                    for another_order in orders:
                        if set(candidate).issubset(another_order):
                            count += 1
                    if count >= 2 and count >= longest_length[length]:
                        order_count[candidate] = count
                        longest_length[length] = count
            else:
                continue
    for k, v in order_count.copy().items():
        if v < longest_length[len(k)]:
            del order_count[k]
    return sorted(order_count)

# 3번

--> 시간초과

In [131]:
import pandas as pd

In [130]:
info = ["java backend junior pizza 150","python frontend senior chicken 210","python frontend senior chicken 150","cpp backend senior pizza 260","java backend junior chicken 80","python backend senior chicken 50"]
query = ["java and backend and junior and pizza 100","python and frontend and senior and chicken 200","cpp and - and senior and pizza 250","- and backend and senior and - 150","- and - and - and chicken 100","- and - and - and - 150"]

In [224]:
import pandas as pd

def parse_info_into_df(info):
    df = pd.DataFrame(columns={"lang", "f/b", "j/s", "food", "score"})
    for applicant in info:
        lang, end, year, food, score = applicant.split(' ')
        df = df.append({"lang": lang, "f/b": end, "j/s": year, "food": food, "score": int(score)}, ignore_index=True)
    return df


def filter_by_condition(df, condition):
    lang, end, year, food_and_score = [x.strip(' ') for x in condition.split('and')]
    food, score = food_and_score.split(' ')
    score = int(score)
    cond1 = (df['lang'] == lang) if lang != '-' else True
    cond2 = (df['f/b'] == end) if end != '-'else True
    cond3 = (df['j/s'] == year) if year != '-'else True
    cond4 = (df['food'] == food) if food != '-'else True
    cond5 = (df['score'] >= score) if score != '-'else True
    
    df = df[cond5]
    selected = df[cond1 & cond2 & cond3 & cond4]
    return len(selected)

def solution(info, query):
    answer = []
    df = parse_info_into_df(info)
    for condition in query:
        answer.append(filter_by_condition(df, condition))
    return answer

In [222]:
[1,1,1,1,2,4]

[1, 1, 1, 1, 2, 4]

# 4번

In [518]:
from collections import defaultdict 
from collections import deque

class Graph: 

    def __init__(self, vertices): 
        # No. of vertices 
        self.V = vertices 
        # default dictionary to store graph 
        self.edges = defaultdict(list)
        self.weights = {}
        self.paths = set()

    # function to add an edge to graph 
    def add_edge(self, from_node, to_node, weight): 
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight

    def get_all_path(self, from_node, to_node, visited, path): 
        visited[from_node-1]= True
        path.append(from_node) 
        
        if from_node == to_node: 
            self.paths.add(tuple(path))
            
        else: 
            for i in self.edges[from_node]: 
                if visited[i-1]== False: 
                    self.get_all_path(i, to_node, visited, path) 
        path.pop() 
        visited[from_node-1]= False
        
        return self.paths
    

def calc_fare(fare, route):
    route = deque(route)
    temp = route.copy()
    total = 0
    s = temp.popleft()
    while len(temp) >= 1:
        d = temp.popleft()
        total += fare[(s, d)]
        s = d
    return total

def check_excerpt_in_routes(excerpt, routes):
    check = False
    matching_routes = []
    for route in routes:
        route = ''.join(map(str, route))
        if excerpt in route:
            check = True
            matching_routes.append(route)
    return check, matching_routes

def solution(n, s, a, b, fares):
    
    graph = Graph(n)
    
    for fare in fares:
        graph.add_edge(*fare)
    
    a_routes = graph.get_all_path(s, a, [False]*graph.V, [])
    graph.paths = set()   #초기화
    b_routes = graph.get_all_path(s, b, [False]*graph.V, [])
    
    possible_fares = []
    
    for route in a_routes:
        route = deque(route)
        route1 = route.copy()
        total = 0
        s = route1.popleft()
        excerpt = str(s)
        check = True
        matching_routes = []
        while check == True and len(route1) >= 1:
            d = route1.popleft()
            excerpt += str(d)
            check, new_matching_routes = check_excerpt_in_routes(excerpt, b_routes)
            if check == False:
                excerpt = excerpt[:-1]
            if len(new_matching_routes) != 0:
                matching_routes = new_matching_routes
            s = d
        
        if len(matching_routes) != 0:   #합승할 때
            #겹치는 부분
            total += calc_fare(graph.weights, map(int, tuple(excerpt)))

            #겹치지 않는 부분
            route = ''.join(map(str, route))
            remaining_a_route = tuple(map(int, route.strip(excerpt[:-1])))
            if len(remaining_a_route) != 0:
                total += calc_fare(graph.weights, remaining_a_route )

            selected_b_route = min(matching_routes)
            remaining_b_route = tuple(map(int, selected_b_route.strip(excerpt[:-1])))
            if len(remaining_b_route) != 0:
                total += calc_fare(graph.weights, remaining_b_route )
            
        else:   #합승하지 않을 때
            total += calc_fare(graph.weights, route)
            total += min(calc_fare(graph.weights, x) for x in b_routes)
    
    
        possible_fares.append(total)
        
    return min(possible_fares)

In [519]:
n = 6
s = 4
a = 6
b = 2
fares = [[4, 1, 10], [3, 5, 24], [5, 6, 2], [3, 1, 41], [5, 1, 24], [4, 6, 50], [2, 4, 66], [2, 3, 22], [1, 6, 25]]

In [520]:
solution(n, s, a, b, fares)

82

In [521]:
solution(7, 3, 4, 1, [[5, 7, 9], [4, 6, 4], [3, 6, 1], [3, 2, 3], [2, 1, 6]])

14